# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [10]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'heart_failure_automl2'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [3]:
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath
import pandas as pd
from azureml.train.automl import AutoMLConfig

dataset = Dataset.get_by_name(ws, name='heart_rate_failure_prediction')  

#x = dataset.to_pandas_dataframe().dropna()
#y = x.pop("HeartDisease")
#dataset = pd.concat([x,y], axis=1)

#pd.DataFrame(dataset).to_csv("data/train_data.csv", index=False)
#ds = ws.get_default_datastore()
#ds.upload(src_dir='./data', target_path='HeartDataSet', overwrite=True, show_progress=True)
# Upload the training data as a tabular dataset for access during training on remote compute
#train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('HeartDataSet/train_data.csv'))
#
#automl_config = AutoMLConfig(    
#    experiment_timeout_minutes=30,
#    enable_onnx_compatible_models=True,
#    task= 'classification',
#    primary_metric= 'AUC_weighted',
#    training_data=dataset,
#    label_column_name='HeartDisease',
#    n_cross_validations=2)

automl_settings = {
    "experiment_timeout_minutes" : 25,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
}

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target="Udacity-Capstone-Project-Cluster",
                             training_data = dataset,
                             label_column_name = "HeartDisease",
                             enable_onnx_compatible_models=True,
                             **automl_settings
                            )  




  







In [5]:
Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

{
    "assetId": null,
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20231030.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "sklearn-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
     

In [12]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heart_failure_automl2,AutoML_a8392b16-812c-479d-97c0-f74e91e2f089,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [16]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

best_aml_model, onnx_model =remote_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, "./automl_model.onnx")


In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
